In [1]:
import pandas as pd
import plotly
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

In [3]:
shared_id_vars = ['Area Code', 'Area', 'Item Code', 'Item', 'Element Code', 'Element', 'Unit']
necessary_id_vars = ['Area', 'Item', 'Element', 'Unit', 'Year', 'value']

In [4]:
df_configs = {
    'df_agr': {
        'filename': 'Value_of_Production_E_All_Area_Groups_NOFLAG.csv',
        'element': 'Gross Production Value (constant 2014-2016 million US$)',
        'value_name': 'Agr_Production_Value'
    },
    'df_emi': {
        'filename': 'Emissions_Agriculture_Agriculture_total_E_All_Area_Groups_NOFLAG.csv',
        'element': 'Emissions (CO2eq)',
        'value_name': 'Emissions_Value'
    },
    'df_inp': {
        'filename': 'Inputs_FertilizersNutrient_E_All_Area_Groups_NOFLAG.csv',
        'element': 'Agricultural Use',
        'value_name': 'Fertilizer_Value'
    }
}

data = {}
for df_name, config in df_configs.items():
    df = pd.read_csv(config['filename'], encoding='latin1')
    df = df.melt(id_vars = shared_id_vars, var_name = 'Year')
    df = df[necessary_id_vars]
    df = df[df.Element == config['element']]
    df = df.rename(columns={'value': config['value_name']})
    data[df_name] = df

In [5]:
df_agr = data['df_agr']
df_emi = data['df_emi']
df_inp = data['df_inp']

In [6]:
# Add subtotals
necessary_inp_columns=['Area', 'Item', 'Element', 'Unit', 'Year', 'Fertilizer_Value']
df_inp_subtotals = pd.DataFrame(columns=necessary_inp_columns)
for area_year, sub_df in df_inp.groupby(['Area', 'Year']):
    subt = pd.DataFrame([[area_year[0], 'Total Fertilizer', 'Agricultural Use', 'tonnes', area_year[1], sub_df.Fertilizer_Value.sum()]], columns=necessary_inp_columns)
    df_inp = df_inp.append(subt)

# Remove projections
projected_years = ['Y2019', 'Y2030', 'Y2050']
df_emi = df_emi[~df_emi.Year.isin(projected_years)]

In [7]:
# select only aggregates
agr_aggregates = ['Agriculture (PIN)', 'Crops (PIN)', 'Livestock (PIN)']
df_agr = df_agr[df_agr.Item.isin(agr_aggregates)]

# change units from $mm to $s
df_agr['Agr_Production_Value'] = df_agr['Agr_Production_Value'] * 10 ** 6
df_agr['Element'] = 'Gross Production Value (constant 2014-2016 US$)'

<ipython-input-7-6512a5b0b428>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agr['Agr_Production_Value'] = df_agr['Agr_Production_Value'] * 10 ** 6
<ipython-input-7-6512a5b0b428>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agr['Element'] = 'Gross Production Value (constant 2014-2016 US$)'


In [8]:
df_combo = df_agr.merge(df_inp, on=['Area', 'Year'])
df_combo = df_combo.merge(df_emi, on=['Area', 'Year'])

In [9]:
df_combo['Agr Production per Fertilizer Inp'] = df_combo['Agr_Production_Value'].div(df_combo['Fertilizer_Value'], axis=0)
df_combo['Agr Production per Emissions'] = df_combo['Agr_Production_Value'].div(df_combo['Emissions_Value'], axis=0)

In [10]:
# Begin plots

In [11]:
def unique_sorted_values(array):
    unique = array.unique().tolist()
    unique.sort()
    return unique

In [12]:
def CAGR(init_year, term_year, init_val, term_val):
    total_growth = term_val / init_val
    num_years = term_year - init_year
    ann_growth = total_growth**(1/num_years)-1
    return ann_growth

In [13]:
# Agriculture production plot

def common_filtering_ag_prod(area):
    plot_output_ag_prod.clear_output()
    
    common_filter_ag_prod = df_agr[df_agr.Area == area]
    
    with plot_output_ag_prod:
        fig_ag_prod = px.line(common_filter_ag_prod, x="Year", y="Agr_Production_Value", color='Item')
        fig_ag_prod.update_layout(yaxis_title='Production Value (2014-2016 USD)',
                      title='Agricultural Production Value')
        fig_ag_prod.show()

def dropdown_area_eventhandler_ag_prod(change):
    common_filtering_ag_prod(change.new)


plot_output_ag_prod = widgets.Output()

# dropdown_area_ag_prod = widgets.Dropdown(options = unique_sorted_values(df_agr.Area))
dropdown_area_ag_prod = widgets.Dropdown(options=sorted(df_agr.Area.unique()))

# how to send dropdown_area_eventhandler with dataframe as argument so I don't need to write these functions for each plot?
dropdown_area_ag_prod.observe(dropdown_area_eventhandler_ag_prod, names='value') 
display(dropdown_area_ag_prod)
display(plot_output_ag_prod)

Dropdown(options=('Africa', 'Americas', 'Asia', 'Australia and New Zealand', 'Caribbean', 'Central America', '…

Output()

Total global agricultural production, including all crops and livestock, grew from $2.8 trillion in 2000 to $4.1 trillion in 2018, a compound annual growth rate of 0.7%

In [14]:
# Fertilizer use plot

def common_filtering_fert(area):
    plot_output_fert.clear_output()
    
    common_filter_fert = df_inp[df_inp.Area == area]
    
    with plot_output_fert:
        fig_fert = px.line(common_filter_fert, x="Year", y="Fertilizer_Value", color='Item')
        fig_fert.update_layout(yaxis_title='Fertilizer (tonnes)',
                      title='Agricultural Fertilizer Use')
        fig_fert.show()

def dropdown_area_eventhandler_fert(change):
    common_filtering_fert(change.new)


plot_output_fert = widgets.Output()

dropdown_area_fert = widgets.Dropdown(options = unique_sorted_values(df_agr.Area))

# how to send dropdown_area_eventhandler with dataframe as argument so I don't need to write these functions for each plot?
dropdown_area_fert.observe(dropdown_area_eventhandler_fert, names='value')
display(dropdown_area_fert)
display(plot_output_fert)

Dropdown(options=('Africa', 'Americas', 'Asia', 'Australia and New Zealand', 'Caribbean', 'Central America', '…

Output()

Total global agricultural use of Nitrogen, Phosphate, and Potash grew at a 1.9% CAGR from 135 million tonnes in 2000 to 188 million tons in 2018. Fertilizer use grew more rapidly than overall agricultural production value, implying increased intensity of fertilizer usage over the past two decades. Use of nintrogen increased most rapidly, with phosphate usage increasing at a relatively lower rate of 0.013%.

In [15]:
# Emissions plot

def common_filtering_emi(area):
    plot_output_emi.clear_output()
    
    common_filter_emi = df_emi[df_emi.Area == area]
    
    with plot_output_emi:
        fig_emi = px.line(common_filter_emi, x="Year", y="Emissions_Value", color='Item')
        fig_emi.update_layout(yaxis_title='Emissions in CO2eq (gigagrams)',
                      title='Agricultural Emissions of CH4 and NO2 in CO2 Equivalent')
        fig_emi.show()

def dropdown_area_eventhandler_emi(change):
    common_filtering_emi(change.new)


plot_output_emi = widgets.Output()

dropdown_area_emi = widgets.Dropdown(options = unique_sorted_values(df_agr.Area))

# how to send dropdown_area_eventhandler with dataframe as argument so I don't need to write these functions for each plot?
dropdown_area_emi.observe(dropdown_area_eventhandler_emi, names='value')
display(dropdown_area_emi)
display(plot_output_emi)

Dropdown(options=('Africa', 'Americas', 'Asia', 'Australia and New Zealand', 'Caribbean', 'Central America', '…

Output()

Agricultural emissions of CH4 and NO2 grew at a 0.007% from 4.7 million gigagrams of CO2 equivalent to 5.3 million Gigagrams of CO2 equivalent (equivalent to 5.3 Gigatonnes). Agricultural emissions come primarily from enteric fermentation and emissions released from agricultural soils, each representing 2.1 million Gigagrams, or 40%, of the 5.3 million Gigagram CO2eq total emissions in 2018.

In [16]:
# Emissions per production plot

def common_filtering_emi_ag(area):
    plot_output_emi_ag.clear_output()
    
    common_filter_emi_ag = df_combo[(df_combo.Area == area) & (df_combo.Item == "Agriculture total")]
    
    with plot_output_emi_ag:
        fig_emi_ag = px.line(common_filter_emi_ag, x="Year", y="Agr Production per Emissions", color='Item_x')
        fig_emi_ag.update_layout(yaxis_title='Agricultural Production (2014-2016 USD) per gigagram of CO2eq Emissions',
                      title='Agricultural Production per gigagram of CO2eq Emissions')
        fig_emi_ag.show()

def dropdown_area_eventhandler_emi_ag(change):
    common_filtering_emi_ag(change.new)


plot_output_emi_ag = widgets.Output()

dropdown_area_emi_ag = widgets.Dropdown(options = unique_sorted_values(df_agr.Area))

# how to send dropdown_area_eventhandler with dataframe as argument so I don't need to write these functions for each plot?
dropdown_area_emi_ag.observe(dropdown_area_eventhandler_emi_ag, names='value')
display(dropdown_area_emi_ag)
display(plot_output_emi_ag)

Dropdown(options=('Africa', 'Americas', 'Asia', 'Australia and New Zealand', 'Caribbean', 'Central America', '…

Output()

Agricultural production has become less emissions intensive on a per dollar basis over the past two decades. Total agricultural production per gigagram of CO2eq emissions increased from approximately $606K in 2000 to $769K in 2018.

In [17]:
# Phosphate per production plot

def common_filtering_phos_ag(area):
    plot_output_phos_ag.clear_output()
    
    common_filter_phos_ag = df_combo[(df_combo.Area == area) & (df_combo.Item_y == "Nutrient phosphate P2O5 (total)")]
    
    with plot_output_phos_ag:
        fig_phos_ag = px.line(common_filter_phos_ag, x="Year", y="Agr Production per Fertilizer Inp", color='Item_x')
        fig_phos_ag.update_layout(yaxis_title='Agricultural Production (2014-2016 USD) per Phosphate P2O5 (tonnes)',
                      title='Agricultural Production per Phosphate Usage')
        fig_phos_ag.show()

def dropdown_area_eventhandler_phos_ag(change):
    common_filtering_phos_ag(change.new)


plot_output_phos_ag = widgets.Output()

dropdown_area_phos_ag = widgets.Dropdown(options = unique_sorted_values(df_agr.Area))

# how to send dropdown_area_eventhandler with dataframe as argument so I don't need to write these functions for each plot?
dropdown_area_phos_ag.observe(dropdown_area_eventhandler_phos_ag, names='value')
display(dropdown_area_phos_ag)
display(plot_output_phos_ag)

Dropdown(options=('Africa', 'Americas', 'Asia', 'Australia and New Zealand', 'Caribbean', 'Central America', '…

Output()

We have also seen an increase in global agricultural production value per tonne of phosphate usage between 2000 and 2018, although most of the increase came between 2016 and 2018, with 2016 having roughly the same value per tonne of phosphate as 2016. 